In [0]:
import pandas as pd#판다스
import requests as re#리퀘스트
from bs4 import BeautifulSoup as bs#뷰티플 소프

from google.colab import drive

In [159]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
titles = ["로봇_원시데이터_281.xlsx", "빅데이터_원시데이터_47.xlsx", "인공지능_원시데이터_28.xlsx", "원시데이터_345.xlsx"]

url = "/content/drive/My Drive/19_Fall/system/2_project/worknet_wanted_detail_final_"
robot = pd.read_excel(url+titles[0], encoding = 'utf-8')
big_data = pd.read_excel(url+titles[1], encoding = 'utf-8')
ai = pd.read_excel(url+titles[2], encoding = 'utf-8')
all_data= pd.read_excel(url+titles[3], encoding = 'utf-8')

In [0]:
#돈 문자열을 숫자열로 바꿔주기
def real_money(value):
  value = value.split(',')
  real_value = ""
  for i in value:
    real_value += i
  return int(real_value)

In [0]:
start_time = []
end_time = []
day_time = []

def set_day_time(all_data):
  for time in all_data['근무시간'] + "*" + all_data['임금조건']:
    if "연봉" in time or "월급" in time or "평균근무시간" in time:
      start_time.append(None)
      end_time.append(None)
      day_time.append(None)
    else:
      if "시 " in time and "분," in time:
        if len(time.split("(")[1].split("오전) ")) >= 2:
          if time.split("(")[1].split("오전) ")[1].split('시')[1].split('분')[0] == ' 00':
            start_time.append(float(time.split("(")[1].split("오전) ")[1].split('시')[0]))
          else:
            start_time.append(float(time.split("(")[1].split("오전) ")[1].split('시')[0]) + 0.5)
        elif len(time.split("(")[1].split("오후) ")) >= 2:
          if time.split("(")[1].split("오후) ")[1].split('시')[1].split('분')[0] == ' 00':
            start_time.append(float(time.split("(")[1].split("오후) ")[1].split('시')[0]) + 12)
          else:
            start_time.append(float(time.split("(")[1].split("오후) ")[1].split('시')[0]) + 12.5)

        if len(time.split("(")[2].split("오전) ")) >= 2:
          if time.split("(")[2].split("오전) ")[1].split('시')[1].split('분')[0] == ' 00':
            end_time.append(float(time.split("(")[2].split("오전) ")[1].split('시')[0]))
          else:
            end_time.append(float(time.split("(")[2].split("오전) ")[1].split('시')[0]) + 0.5)
        elif len(time.split("(")[2].split("오후) ")) >= 2:
          if time.split("(")[2].split("오후) ")[1].split('시')[1].split('분')[0] == ' 00':
            end_time.append(float(time.split("(")[2].split("오후) ")[1].split('시')[0]) + 12)
          else:
            end_time.append(float(time.split("(")[2].split("오후) ")[1].split('시')[0]) + 12.5) 
        elif len(time.split("(")[2].split("정오) ")) >= 2:
          if time.split("(")[2].split("정오) ")[1].split('시')[1].split('분')[0] == ' 00':
            end_time.append(float(time.split("(")[2].split("정오) ")[1].split('시')[0]) + 12 if time.split("(")[2].split("정오) ")[1].split('시')[0] != '12' else 12)
          else:
            end_time.append(float(time.split("(")[2].split("정오) ")[1].split('시')[0]) + 12.5)
        elif len(time.split("(")[2].split("오후)")) >= 2:
          if time.split("(")[2].split("오후)")[1].split('시')[1].split('분')[0] == ' 00':
            end_time.append(float(time.split("(")[2].split("오후)")[1].split('시')[0]) + 12 if time.split("(")[2].split("오후)")[1].split('시')[0] != '12' else 12)
          else:
            end_time.append(float(time.split("(")[2].split("오후)")[1].split('시')[0]) + 12.5)

        
        data = start_time[len(end_time) - 1] - end_time[len(end_time) - 1]
        if data > 0:
          day_time.append(data)
        else:
          day_time.append(-data)

In [0]:
#하루 일 시작 시간, 하루 일 끝나는 시간, 하루 동안 일하는 시간, 한주에 일하는 시간, 주당 일하는 시간

week_time = [] #주당 일하는 시간
week_day = [] #주당 일하는 날수

def set_time(all_data):
  count = 0
  for data in all_data['근무시간']:
    if len(data.split(", 주 ")) == 2:
      week_day.append(data.split(", 주 ")[1].split('일')[0])
    else:
      week_day.append(None)

    if data.split(" ").count('평균근무시간') == 1:
      week_time.append(data.split("평균근무시간 : ")[1])
    else:
      if day_time[count] != None and week_day[count] != None:
        week_time.append(float(day_time[count]) * float(week_day[count]))
    count+=1

In [0]:
# 최저 연봉
data_low = []
def set_data_low(all_data):
  count = 0

  for data in all_data['임금조건']:
    #최저 연봉
    if data[0] == '연':
      value = data.split('연봉')[1].split('원 이상')[0]
      value = int(real_money(value))
    elif data[0] == '월':
      value = data.split('월급')[1].split('원 이상')[0]
      value = int(real_money(value)) * 12
    elif data[0] == '일':
      value = data.split('일급')[1].split('원 이상')[0]

      if week_day[count] != None:
        value = int(real_money(value)) * 4 * int(week_day[count]) * 12

    elif data[0] == '시':
      value = data.split('시급')[1].split('원 이상')[0]
      if week_time[count] != None:
        value = int(real_money(value)) * 4 * int(week_time[count]) * 12
      elif week_day[count] != None and day_time[count] != None:
        value = int(real_money(value)) * 4 * int(week_day[count]) * day_time[count] * 12 # 하루 근무 시간 삽입 8


    count+=1
    data_low.append(value)

In [0]:
#최대 연봉
data_high = []
def set_data_high(all_data):
  count = 0

  for data in all_data['임금조건']:
    #최고 연봉
    if len(data.split(" ~ ")) == 2:
      value = data.split(" ~ ")[1].split("원 이하")[0]
      if data[0] == '연':
        value = int(real_money(value))
      elif data[0] == '월':
        value = int(real_money(value)) * 12
      elif data[0] == '시':
        if week_time[count] != None:
          value = int(real_money(value)) * 4 * int(week_time[count]) * 12
        elif week_day[count] != None:
          value = int(real_money(value)) * 4 * int(week_day[count]) * day_time[count] * 12
      elif data[0] == '일':
        if week_day[count] != None:
          value = int(real_money(value)) * 4 * int(week_day[count]) * 12
        else:
          value = int(real_money(value)) * 4 * 5 * 12
      #print(str(count) + ' ' + str(value))
      data_high.append(value)
    else:
      data_high.append(data_low[count])
    count += 1

In [0]:
data_mid = []
def set_data_mid(all_data):
  for i in range(len(data_high)):
    data_mid.append(int((data_high[i]+data_low[i])/2))
    #print(str(data_low[i]) + " " + str(data_high[i]) + " " + str(data_mid[i]))

In [0]:
data_loc = []
def set_data_loc(all_data):
  for data in all_data["근무예정지"]:
    data_locs = data.split(' ')
    data_loc.append(data_locs[2])
  #all_data[['근무예정지']] = data_loc


In [0]:
def set_all_data(all_data):
  set_day_time(all_data)
  set_time(all_data)
  set_data_low(all_data)
  set_data_high(all_data)
  set_data_mid(all_data)
  set_data_loc(all_data)

In [0]:
def set_print_all_data(robot):
  set_all_data(robot)
  make_data = {}
  make_data['start_time'] = start_time
  make_data['end_time'] = end_time
  make_data['day_time'] = day_time
  make_data['week_time'] = week_time
  make_data['week_day'] = week_day
  make_data['data_low'] = data_low
  make_data['data_high'] = data_high
  make_data['data_mid'] = data_mid
  make_data['data_loc'] = data_loc
  my_df = pd.DataFrame(make_data)
  return my_df

In [0]:
start_time = []
end_time = []
day_time = []
week_time = []
week_day = []
data_low = []
data_high = []
data_mid = []
data_loc = []
ai_make_data = set_print_all_data(ai)
start_time = []
end_time = []
day_time = []
week_time = []
week_day = []
data_low = []
data_high = []
data_mid = []
data_loc = []
all_data_make_data = set_print_all_data(all_data)
start_time = []
end_time = []
day_time = []
week_time = []
week_day = []
data_low = []
data_high = []
data_mid = []
data_loc = []
big_data_make_data = set_print_all_data(big_data)
start_time = []
end_time = []
day_time = []
week_time = []
week_day = []
data_low = []
data_high = []
data_mid = []
data_loc = []
robot_make_data = set_print_all_data(robot)

In [0]:
#robot, ai, big_data, all_data, robot_make_data, ai_make_data, big_data_make_data, all_data_make_data

In [0]:
def set_woker(all_data):
  worker = []
  count = 0
  for data in all_data['근로자수']:
    worker.append((int)(data.split(" 명")[0]))
    count+=1
  return worker

In [0]:
make_data = {}
make_data['로봇'] = set_woker(robot)
make_data['빅데이터'] = set_woker(big_data)
make_data['인공지능'] = set_woker(ai)

my_df = pd.concat([pd.DataFrame(make_data['로봇']), pd.DataFrame(make_data['빅데이터']), pd.DataFrame(make_data['인공지능'])],axis = 1)

my_df.loc["sum"] = my_df.sum()
my_df = my_df.append(my_df.mode().drop(my_df.mode().index[1]).rename(index={0: 'mode'}))

my_df.columns = ['로봇', '빅데이터', '인공지능']

url = "/content/drive/My Drive/19_Fall/system/2_project/"
my_df.to_excel(url + "분야별 각각의 근로자 수" + ".xlsx")

In [0]:
make_data = {}
make_data['로봇'] = robot['모집인원']
make_data['빅데이터'] = big_data['모집인원']
make_data['인공지능'] = ai['모집인원']

my_df = pd.concat([pd.DataFrame(make_data['로봇']), pd.DataFrame(make_data['빅데이터']), pd.DataFrame(make_data['인공지능'])],axis = 1)

my_df.loc["sum"] = my_df.sum()
my_df = my_df.append(my_df.mode().drop(my_df.mode().index[1]).rename(index={0: 'mode'}))

my_df.columns = ['로봇', '빅데이터', '인공지능']

url = "/content/drive/My Drive/19_Fall/system/2_project/"
my_df.to_excel(url + "분야별 각각의 모집인원 수" + ".xlsx")

In [0]:
make_data = {}
make_data['로봇최저연봉'] = robot_make_data['data_low']
make_data['로봇평균연봉'] = robot_make_data['data_mid']
make_data['로봇최고연봉'] = robot_make_data['data_high']

make_data['빅데이터최저연봉'] = big_data_make_data['data_low']
make_data['빅데이터평균연봉'] = big_data_make_data['data_mid']
make_data['빅데이터최고연봉'] = big_data_make_data['data_high']

make_data['인공지능최저연봉'] = ai_make_data['data_low']
make_data['인공지능평균연봉'] = ai_make_data['data_mid']
make_data['인공지능최고연봉'] = ai_make_data['data_high']

my_df = pd.DataFrame(make_data)
my_df.loc["sum"] = my_df.sum()
my_df =my_df.append(my_df.mode().rename(index={0: 'mode'}))

url = "/content/drive/My Drive/19_Fall/system/2_project/"
my_df.to_excel(url + "분야별 각각의 임금(최저 평균 최고)" + ".xlsx")

In [0]:
make_data = {}
make_data['로봇 공고수'] = [len(robot)]
make_data['빅데이터 공고수'] = [len(big_data)]
make_data['인공지능 공고수'] = [len(ai)]

my_df = pd.DataFrame(make_data)

url = "/content/drive/My Drive/19_Fall/system/2_project/"
my_df.to_excel(url + "분야별 공고수" + ".xlsx")

In [0]:
def newValue(data):
  return pd.Series(data['업종'].value_counts())

In [0]:
make_data = {}
make_data['로봇 업종'] = newValue(robot)
make_data['빅데이터 업종'] = newValue(big_data)
make_data['인공지능 업종'] = newValue(ai)

df = pd.DataFrame(make_data)

url = "/content/drive/My Drive/19_Fall/system/2_project/"
df.to_excel(url + "분야별 업종수" + ".xlsx")

In [0]:
def newComSize(data):
  return pd.Series(data['회사규모'].value_counts())

In [0]:
make_data = {}
make_data['로봇 회사규모'] = newComSize(robot)
make_data['빅데이터 회사규모'] = newComSize(big_data)
make_data['인공지능 회사규모'] = newComSize(ai)

df = pd.DataFrame(make_data)

url = "/content/drive/My Drive/19_Fall/system/2_project/"
df.to_excel(url + "분야별 회사규모" + ".xlsx")

In [0]:
def newJob(data):
  return pd.Series(data['모집집종'].value_counts())

In [0]:
make_data = {}
make_data['로봇 모집집종'] = newJob(robot)
make_data['빅데이터 모집집종'] = newJob(big_data)
make_data['인공지능 모집집종'] = newJob(ai)

df = pd.DataFrame(make_data)
print(df)
url = "/content/drive/My Drive/19_Fall/system/2_project/"
df.to_excel(url + "분야별 모집집종" + ".xlsx")

In [0]:
def endMoney(data):
  return pd.Series(data['퇴직금'].value_counts())

In [0]:
make_data = {}
make_data['로봇 퇴직금'] = endMoney(robot)
make_data['빅데이터 퇴직금'] = endMoney(big_data)
make_data['인공지능 퇴직금'] = endMoney(ai)

df = pd.DataFrame(make_data)
print(df)
url = "/content/drive/My Drive/19_Fall/system/2_project/"
df.to_excel(url + "분야별 퇴직금" + ".xlsx")

In [0]:
make_data = {}
make_data['로봇 근무예정지'] = robot_make_data['data_loc']
make_data['빅데이터 근무예정지'] = big_data_make_data['data_loc']
make_data['인공지능 근무예정지'] = ai_make_data['data_loc']

my_df = pd.DataFrame(make_data)

make_data = {}
make_data['근무예정지'] = my_df.groupby("로봇 근무예정지").size().index
make_data['로봇 수'] = my_df.groupby("로봇 근무예정지").size().values
data = pd.DataFrame(make_data)

make_data = {}
make_data['근무예정지'] = my_df.groupby("빅데이터 근무예정지").size().index
make_data['빅데이터 수'] = my_df.groupby("빅데이터 근무예정지").size().values
data2 = pd.DataFrame(make_data)

data = pd.merge(data, data2, on = '근무예정지', how = "outer")

make_data = {}
make_data['근무예정지'] = my_df.groupby("인공지능 근무예정지").size().index
make_data['인공지능 수'] = my_df.groupby("인공지능 근무예정지").size().values
data2 = pd.DataFrame(make_data)

data = pd.merge(data, data2, on = '근무예정지', how = "outer")

data = data.fillna(value=0)

#print(pd.merge(my_df.groupby('로봇 근무예정지').size(), my_df.groupby('빅데이터 근무예정지').size(), \
#              my_df.groupby('인공지능 근무예정지').size()))

url = "/content/drive/My Drive/19_Fall/system/2_project/"
my_df.to_excel(url + "분야별 근무예정지" + ".xlsx")
data.to_excel(url + "분야별 근무예정지 인원수.xlsx")

In [0]:
def gettype(data):
  return pd.Series(data['고용형태'].value_counts())

In [0]:
make_data = {}
make_data['로봇 고용형태'] = gettype(robot)
make_data['빅데이터 고용형태'] = gettype(big_data)
make_data['인공지능 고용형태'] = gettype(ai)

df = pd.DataFrame(make_data)
print(df)
url = "/content/drive/My Drive/19_Fall/system/2_project/"
df.to_excel(url + "분야별 고용형태" + ".xlsx")

In [0]:
def gettype(data):
  return pd.Series(data['경력조건'].value_counts())

In [0]:
make_data = {}
make_data['로봇 경력조건'] = gettype(robot)
make_data['빅데이터 경력조건'] = gettype(big_data)
make_data['인공지능 경력조건'] = gettype(ai)

df = pd.DataFrame(make_data)
print(df)
url = "/content/drive/My Drive/19_Fall/system/2_project/"
df.to_excel(url + "분야별 경력조건" + ".xlsx")

In [0]:
make_data = {}
make_data['로봇 최저 학력'] = robot["최소학력코드"]
make_data['로봇 최대 학력'] = robot["최대학력코드"]

make_data['빅데이터 최저 학력'] = big_data["최소학력코드"]
make_data['빅데이터 최대 학력'] = big_data["최대학력코드"]

make_data['인공지능 최저 학력'] = ai["최소학력코드"]
make_data['인공지능 최대 학력'] = ai["최대학력코드"]

my_df = pd.DataFrame(make_data)

my_df =my_df.append(my_df.mode().rename(index={0: 'mode'}))

url = "/content/drive/My Drive/19_Fall/system/2_project/"
my_df.to_excel(url + "분야별 각각의 학력(최소 최대) + 최하단 평균값 기재" + ".xlsx")

In [0]:
value = pd.concat([all_data, all_data_make_data], axis = 1)

value = value.sort_values(by = "data_mid")
value = value.reset_index()
value.drop('index', axis=1, inplace=True)
value.drop('Unnamed: 0', axis=1, inplace=True)

slice_n = 20000000
values = []
values.append(value.loc[value['data_mid'] < slice_n, :])
while slice_n != 50000000:
  slice_n += 5000000
  temp = value.loc[value['data_mid'] < slice_n, :]
  values.append(temp.loc[slice_n - 5000000 <= value['data_mid'], : ])
values.append(value.loc[slice_n <= value['data_mid'], : ])
for i in range(7):
  values[i] = values[i].reset_index()
  values[i].drop('index', axis=1, inplace=True)
#value.to_excel(url + "토탈데이터.xlsx")

In [0]:
make_data = {}
"""
make_data['2천만 미만'] = values[0]
make_data['2천만 이상 2천5백만 미만'] = values[1]
make_data['2천5백만 이상 3천만 미만'] = values[2]
make_data['3천만 이상 3천5백만 미만'] = values[3]
make_data['3천5백만 이상 4천만 미만'] = values[4]
make_data['4천만 이상 4천5백만 미만'] = values[5]
make_data['4천5백만 이상'] = values[6]
"""
for i in range(7):
  make_data[i] = values[i]['data_loc']

my_df = pd.DataFrame(make_data)

make_data = {}
make_data['근무예정지'] = my_df.groupby(0).size().index
make_data['2천만 미만'] = my_df.groupby(0).size().values
data = pd.DataFrame(make_data)


make_data = {}
make_data['근무예정지'] = my_df.groupby(1).size().index
make_data['2천만 이상 2천5백만 미만'] = my_df.groupby(1).size().values
data2 = pd.DataFrame(make_data)
data = pd.merge(data, data2, on = '근무예정지', how = "outer")

make_data = {}
make_data['근무예정지'] = my_df.groupby(2).size().index
make_data['2천5백만 이상 3천만 미만'] = my_df.groupby(2).size().values
data2 = pd.DataFrame(make_data)
data = pd.merge(data, data2, on = '근무예정지', how = "outer")

make_data = {}
make_data['근무예정지'] = my_df.groupby(3).size().index
make_data['3천만 이상 3천5백만 미만'] = my_df.groupby(3).size().values
data2 = pd.DataFrame(make_data)
data = pd.merge(data, data2, on = '근무예정지', how = "outer")

make_data = {}
make_data['근무예정지'] = my_df.groupby(4).size().index
make_data['3천5백만 이상 4천만 미만'] = my_df.groupby(4).size().values
data2 = pd.DataFrame(make_data)
data = pd.merge(data, data2, on = '근무예정지', how = "outer")

make_data = {}
make_data['근무예정지'] = my_df.groupby(5).size().index
make_data['4천만 이상 4천5백만 미만'] = my_df.groupby(5).size().values
data2 = pd.DataFrame(make_data)
data = pd.merge(data, data2, on = '근무예정지', how = "outer")

make_data = {}
make_data['근무예정지'] = my_df.groupby(6).size().index
make_data['4천5백만 이상'] = my_df.groupby(6).size().values
data2 = pd.DataFrame(make_data)
data = pd.merge(data, data2, on = '근무예정지', how = "outer")

data = data.fillna(value=0)

url = "/content/drive/My Drive/19_Fall/system/2_project/"
my_df.to_excel(url + "임금별 근무예정지" + ".xlsx")
data.to_excel(url + "임금별 근무예정지 인원수.xlsx")

In [0]:
make_data = {}
"""
make_data['2천만 미만'] = values[0]
make_data['2천만 이상 2천5백만 미만'] = values[1]
make_data['2천5백만 이상 3천만 미만'] = values[2]
make_data['3천만 이상 3천5백만 미만'] = values[3]
make_data['3천5백만 이상 4천만 미만'] = values[4]
make_data['4천만 이상 4천5백만 미만'] = values[5]
make_data['4천5백만 이상'] = values[6]
"""
columns = ['2천만 미만', '2천만 이상 2천5백만 미만', '2천5백만 이상 3천만 미만', '3천만 이상 3천5백만 미만'\
          ,'3천5백만 이상 4천만 미만', '4천만 이상 4천5백만 미만', '4천5백만 이상']
for i in range(7):
  make_data[columns[i] + " 최소"] = values[i]['최소학력코드']
  make_data[columns[i] + " 최대"] = values[i]['최대학력코드']

my_df = pd.DataFrame(make_data)

my_df.loc["sum"] = my_df.sum()
my_df =my_df.append(my_df.mode().rename(index={0: 'mode'}))

url = "/content/drive/My Drive/19_Fall/system/2_project/"
my_df.to_excel(url + "임금별 각각의 학력(최소 최대) + 최하단 평균값 기재" + ".xlsx")

In [0]:
value = pd.concat([all_data, all_data_make_data], axis = 1)

value = value.sort_values(by = "최대학력코드")
value = value.reset_index()
value.drop('index', axis=1, inplace=True)
value.drop('Unnamed: 0', axis=1, inplace=True)

slice_n = 0
values = {}
values[slice_n] = value.loc[value['최대학력코드'] == slice_n, :]

for slice_n in range(3, 8):
  temp = value.loc[value['최대학력코드'] == slice_n, :]
  values[slice_n] = temp

values[0] = values[0].reset_index()
values[0].drop('index', axis=1, inplace=True)

for i in range(3, 8):
  values[i] = values[i].reset_index()
  values[i].drop('index', axis=1, inplace=True)
  